# Advanced Lane Lines

### Initial Camera Calibration

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
import glob
from CameraOperations import show_grid, correct_distortion, calibrate_camera_from_folder
import os
import pickle

print("Calibrating camera on images in camera_cal...")
%time mtx, dist = calibrate_camera_from_folder("./camera_cal",name_pattern="calibration*.jpg", counts=(9,6))
print("Done Calibrating.")
global_mtx = mtx
global_dist = dist

print("Correcting calibration images...")
test_images = os.listdir("camera_cal/")
for img in test_images:
    cal_image = mpimg.imread('camera_cal/'+img)
    cal_un_dst = correct_distortion(cal_image, global_mtx, global_dist)
    mpimg.imsave("corrected_images/adj_"+img, cal_un_dst)

# Correct the test images
print("Correcting test images...")
test_images = os.listdir("test_images/")
for img in test_images:
    image = mpimg.imread('test_images/'+img)
    un_dst = correct_distortion(image, global_mtx, global_dist)
    mpimg.imsave("corrected_images/adj_"+img, un_dst)
    
show_grid((cal_image, cal_un_dst), ("Orginal", "Corrected"), ticks=(320,80))
show_grid((image, un_dst), ("Test Image", "Corrected Test Image"), ticks=(320,80))
#global_mtx, global_dist

#Save a dictionary to a pickle file
pickle_dict = {'mtx': global_mtx, 'dist': global_dist}
pickle.dump(pickle_dict, open("./camera_parameters.p", "wb"))

In [ ]:
image2 = mpimg.imread("./camera_cal/calibration2.jpg")
image3 = mpimg.imread("./camera_cal/calibration3.jpg")
image2_undst = mpimg.imread("./corrected_images/adj_calibration2.jpg")
image3_undst = mpimg.imread("./corrected_images/adj_calibration3.jpg")

show_grid((image2, image2_undst), ("Orginal", "Corrected"), ticks=(320,80))
show_grid((image3, image3_undst), ("Original", "Corrected"), ticks=(320,80))


### Test on multiple images

In [ ]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
import glob
from CameraOperations import show_grid, correct_distortion, perspective_transform, sobel_LR_threshold
from LaneOperations import map_lane_lines
from ImageProcessing import image_gradient, image_preprocessing, gaussian_blur, image_sharpen
import pickle

#Load a pickle file
use_dict = True
if use_dict:
    with open("./camera_parameters.p", mode='rb') as f:
        camera_params = pickle.load(f)
    global_dist = camera_params['dist']
    global_mtx = camera_params['mtx']
#viewport = [[540,465],[740,465],[1280,720],[0,720]]   # New Viewport (vanishing point intersection)
#viewport = [[560,455],[720,455],[1280,720],[0,720]]   # New Viewport (vanishing point intersection)
#viewport = [[565,455],[715,455],[1280,720],[0,720]]   # New Viewport (vanishing point intersection)
#viewport = [[488,490],[792,490],[1280,720],[0,720]]   # New Viewport (vanishing point intersection)
viewport = [[570,450],[710,450],[1280,720],[0,720]]   # New Viewport (vanishing point intersection)
viewport = [[574,450],[706,450],[1280,720],[0,720]]   # New Viewport (vanishing point intersection)
offset = 80
sobel_thresh=(30, 150)
gradient_thresh=(0.7, 1.3)
sobel_kernel=15
s_thresh=(170,240)

#images = glob.glob("corrected_images/adj_test*.jpg")
images2 = glob.glob("corrected_images/adj_straight*.jpg")
#images = images+images2
for img in images2:
    image = mpimg.imread(img)
    xform = perspective_transform(image, viewport, offset=80)
    for k in range(len(viewport)):
        x1, y1 = viewport[k][0],viewport[k][1]
        x2, y2 = viewport[k-1][0],viewport[k-1][1]
        cv2.line(image,(x1,y1),(x2,y2), [255,0,0], 2)
    #mapped_lanes, lane_markings,cache = map_lane_lines(image,window_width=50,window_height=80,margin=50) #,DEBUG=True,DEBUG_ID=img)
    show_grid((image, xform), ("Original","Transform"), ticks=(160,48))
    #show_grid((mapped_lanes, lane_markings), ("Mapped Lanes", "Lane Markings"), ticks=(160,48))


In [ ]:
plt.imshow(lane_markings)
mpimg.imsave("./viewport.jpg", image)

### Try it on video

In [ ]:
import imageio
#imageio.plugins.ffmpeg.download()
from moviepy.editor import VideoFileClip
from IPython.display import HTML

idx=0
cache = None
def make_frame_lane_markings(image):
    global idx
    global cache
    DEBUG = True
    un_dst = correct_distortion(image, global_mtx, global_dist)
    mapped_lanes, lane_markings, cache = map_lane_lines(un_dst, window_width=40, window_height=48, margin=40, cache=cache)
    if DEBUG:
        mpimg.imsave("debug_images/{0}_overlay.jpg".format(idx), lane_markings)
        mpimg.imsave("debug_images/{0}_mapped_lanes.jpg".format(idx), mapped_lanes)
        mpimg.imsave("debug_images/{0}.jpg".format(idx), un_dst)
    idx = idx + 1
    return lane_markings

clip_output = "./marked_v5_project_video.mp4" # marked_v4_harder_challenge
clip_handle = VideoFileClip("project_video.mp4", audio=False)
img_clip = clip_handle.fl_image(make_frame_lane_markings) 
%time img_clip.write_videofile(clip_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(clip_output))

### Trying various filters to find the best match

In [ ]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import numpy as np

from CameraOperations import sobel_threshold, gradient_threshold, sobel_and_gradient, sobel_LSR_threshold, show_grid
from CameraOperations import threshold_rgb, threshold_hls, sobel_LR_threshold
from ImageProcessing import image_preprocessing, augment_brightness_camera_images, image_sharpen
from LaneOperations import perspective_transform

#img_id = 206
#image = mpimg.imread("./debug_images3/{0}.jpg".format(img_id))

img_id = "1260"
img_id = "1045"
image = mpimg.imread("./debug_images/{0}.jpg".format(img_id))
#image = mpimg.imread("signs_vehicles_xygrad.png")
#image = mpimg.imread("corrected_images/adj_test4.jpg")
#image = image_sharpen(image)
w = image.shape[1]
h = image.shape[0]
viewport = [[540,465],[740,465],[1280,720],[0,720]]
viewport = [[593,449],[691,449],[1080,720],[204,720]]
#viewport = [[round(w/2-105),round(h*.65)],[round(w/2+105),round(h*.65)],[w,h],[0,h]]
#image = perspective_transform(image, viewport=viewport, offset=0, reverse=False)
r = image[:,:,0]
hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
s = hls[:,:,2]
l = hls[:,:,1]
offset=320

r_v = perspective_transform(r, viewport=viewport, offset=offset, reverse=False)
l_v = perspective_transform(l, viewport=viewport, offset=offset, reverse=False)
s_v = perspective_transform(s, viewport=viewport, offset=offset, reverse=False)

show_grid((r_v,s_v,l_v), ("Red", "Saturation", "Luminosity"))

sobel_thresh=(30, 150)
gradient_thresh=(0.7, 1.3)
sobel_kernel=15

def gradient_threshold2(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    if len(img.shape) == 3:
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    else:
        gray = img
    sobelx = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel))
    sobely = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel))
    gradient = np.arctan2(sobely, sobelx)
    print(np.min(gradient), np.max(gradient))
    sxbinary = np.zeros_like(gradient)
    sxbinary[(gradient >= thresh[0]) & (gradient <= thresh[1])] = 1
    return sxbinary

def sobel_and_gradient2(img, sobel_kernel=3, sobel_thresh=(80, 160), gradient_thresh=(0, np.pi/2)):
    gradx = sobel_threshold(img, orient='x', sobel_kernel=sobel_kernel, thresh=sobel_thresh)
    grady = sobel_threshold(img, orient='y', sobel_kernel=sobel_kernel, thresh=sobel_thresh)
    mag_binary = sobel_threshold(img, orient='both', sobel_kernel=sobel_kernel, thresh=sobel_thresh)
    dir_binary = gradient_threshold2(img, sobel_kernel=sobel_kernel, thresh=gradient_thresh)
    combined = np.zeros_like(dir_binary)
    combined[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1))] = 1
    return combined

DEBUG2 = True
if DEBUG2:
    sobel_r = sobel_threshold(r, orient='y', sobel_kernel=sobel_kernel, thresh=sobel_thresh)
    sobel_s = sobel_threshold(s, orient='y', sobel_kernel=sobel_kernel, thresh=sobel_thresh)
    sobel_l = sobel_threshold(l, orient='y', sobel_kernel=sobel_kernel, thresh=sobel_thresh)
    sobel_r = perspective_transform(sobel_r, viewport=viewport, offset=offset, reverse=False)
    sobel_s = perspective_transform(sobel_s, viewport=viewport, offset=offset, reverse=False)
    sobel_l = perspective_transform(sobel_l, viewport=viewport, offset=offset, reverse=False)
    show_grid((sobel_r,sobel_s,sobel_l), ("Sobel(Y) Red", "Sobel(Y) Saturation", "Sobel(Y) Luminosity"))

    sobel_r = sobel_threshold(r, orient='x', sobel_kernel=sobel_kernel, thresh=sobel_thresh)
    sobel_s = sobel_threshold(s, orient='x', sobel_kernel=sobel_kernel, thresh=sobel_thresh)
    sobel_l = sobel_threshold(l, orient='x', sobel_kernel=sobel_kernel, thresh=sobel_thresh)
    sobel_r = perspective_transform(sobel_r, viewport=viewport, offset=offset, reverse=False)
    sobel_s = perspective_transform(sobel_s, viewport=viewport, offset=offset, reverse=False)
    sobel_l = perspective_transform(sobel_l, viewport=viewport, offset=offset, reverse=False)
    show_grid((sobel_r,sobel_s,sobel_l), ("Sobel(X) Red", "Sobel(X) Saturation", "Sobel(X) Luminosity"))

    sobel_r = sobel_threshold(r, orient='both', sobel_kernel=sobel_kernel, thresh=sobel_thresh)
    sobel_s = sobel_threshold(s, orient='both', sobel_kernel=sobel_kernel, thresh=sobel_thresh)
    sobel_l = sobel_threshold(l, orient='both', sobel_kernel=sobel_kernel, thresh=sobel_thresh)
    sobel_r = perspective_transform(sobel_r, viewport=viewport, offset=offset, reverse=False)
    sobel_s = perspective_transform(sobel_s, viewport=viewport, offset=offset, reverse=False)
    sobel_l = perspective_transform(sobel_l, viewport=viewport, offset=offset, reverse=False)
    show_grid((sobel_r,sobel_s,sobel_l), ("Sobel(Both) Red", "Sobel(Both) Saturation", "Sobel(Both) Luminosity"))

grad_r = gradient_threshold(r, sobel_kernel=sobel_kernel, thresh=gradient_thresh)
grad_l = gradient_threshold(l, sobel_kernel=sobel_kernel, thresh=gradient_thresh)
grad_s = gradient_threshold(s, sobel_kernel=sobel_kernel, thresh=gradient_thresh)
show_grid((grad_r,grad_s,grad_l), ("Gradient Red", "Gradient Saturation", "Gradient Luminosity"))

sg_r = sobel_and_gradient(r, sobel_kernel=sobel_kernel, sobel_thresh=sobel_thresh, gradient_thresh=gradient_thresh)
sg_l = sobel_and_gradient(l, sobel_kernel=sobel_kernel, sobel_thresh=sobel_thresh, gradient_thresh=gradient_thresh)
sg_s = sobel_and_gradient(s, sobel_kernel=sobel_kernel, sobel_thresh=sobel_thresh, gradient_thresh=gradient_thresh)

sg_r = perspective_transform(sg_r, viewport=viewport, offset=offset, reverse=False)
sg_l = perspective_transform(sg_l, viewport=viewport, offset=offset, reverse=False)
sg_s = perspective_transform(sg_s, viewport=viewport, offset=offset, reverse=False)
show_grid((sg_r,sg_s,sg_l), ("Sobel & Gradient Red", "Sobel & Gradient Saturation", "Sobel & Gradient Luminosity"))

t_r = sobel_LR_threshold(image, sobel_kernel=15, sobel_thresh=(30, 150))
t_l = sobel_LR_threshold(image, sobel_kernel=15, sobel_thresh=(30, 150))
t_s = sobel_LR_threshold(image, sobel_kernel=15, sobel_thresh=(30, 150))
t_r = perspective_transform(t_r, viewport=viewport, offset=offset, reverse=False)
t_l = perspective_transform(t_l, viewport=viewport, offset=offset, reverse=False)
t_s = perspective_transform(t_s, viewport=viewport, offset=offset, reverse=False)
show_grid((t_r, t_s, t_l), ("Threshold Red", "Threshold Saturation", "Threshold Luminosity"))


In [ ]:
window_width = 100
window = np.ones(window_width)
l_sum = np.sum(sg_s[int(3*h/4):,:int(w/2)], axis=0)
r_sum = np.sum(sg_s[int(3*h/4):,int(w/2):], axis=0)
l_sum = np.sum(sg_s[int(3*h/4):,:], axis=0)
show_grid((sg_s[int(3*h/4):,:int(w/2)], sg_s[int(3*h/4):,int(w/2):]), ("Left", "Right"))
l_conv_signal = np.convolve(window, l_sum)

#what pixels are non zero in the convolution signal
indicies = np.arange(len(l_conv_signal))
idx_list = np.extract((l_conv_signal > 0) , indicies)
print(len(idx_list), len(l_conv_signal))
plt.figure()
plt.hist(idx_list, normed=False, alpha=0.2, bins=739)
print("Dirty",np.std(idx_list))

l_sum = np.sum(t_s[int(3*h/4):,:int(w/2)], axis=0)
r_sum = np.sum(t_s[int(3*h/4):,int(w/2):], axis=0)
l_sum = np.sum(t_s[int(3*h/4):,:], axis=0)
show_grid((t_s[int(3*h/4):,:], t_s[int(3*h/4):,:]), ("Left", "Right"))
l_conv_signal = np.convolve(window, l_sum)

indicies = np.arange(len(l_conv_signal))
#condition = np.logical_and(l_conv_signal > 0, labels < value+scale)
idx_list = np.extract((l_conv_signal > 0) , indicies)
print(len(idx_list), len(l_conv_signal))
plt.figure()
q = plt.hist(idx_list, normed=False, alpha=0.2, bins=739)
print("clean ",np.std(idx_list))
#show_grid((l_sum,l_conv_signal), ("Sum","Convolution"))


In [ ]:
import numpy as np
import matplotlib.image as mpimg
from CameraOperations import show_grid, sobel_and_gradient
import cv2
%matplotlib inline

sobel_thresh=(30, 150)
gradient_thresh=(0.7, 1.3)
sobel_kernel=9

#img = mpimg.imread("corrected_images/adj_test4.jpg")
img = mpimg.imread("corrected_images/adj_straight_lines1.jpg")
r = img[:,:,0]
hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
s = hls[:,:,2]
l = hls[:,:,1]
hls[:,:,0] = 0

s = sobel_and_gradient(s, sobel_kernel=sobel_kernel, sobel_thresh=sobel_thresh, gradient_thresh=gradient_thresh)
l = sobel_and_gradient(l, sobel_kernel=sobel_kernel, sobel_thresh=sobel_thresh, gradient_thresh=gradient_thresh)
hls = sobel_and_gradient(hls, sobel_kernel=sobel_kernel, sobel_thresh=sobel_thresh, gradient_thresh=gradient_thresh)

combined = np.zeros_like(s).astype("uint8")
combined[((s == 1) | (l == 1))] = 1

s = cv2.cvtColor(s.astype("uint8"), cv2.COLOR_GRAY2RGB)

s = perspective_transform(255*s, viewport=viewport, offset=80, reverse=False)
l = perspective_transform(255*l, viewport=viewport, offset=80, reverse=False)
c = perspective_transform(255*combined, viewport=viewport, offset=80, reverse=False)
hls = perspective_transform(255*hls, viewport=viewport, offset=80, reverse=False)

show_grid((s, l, c, hls), ("s", "l", "combined", "LS"))


In [ ]:
from LaneLines import Road
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from CameraOperations import show_grid, sobel_LS_threshold
%matplotlib inline

sobel_thresh=(30, 150)
gradient_thresh=(0.7, 1.3)
sobel_kernel=9

road = Road(window_height=48, window_width=100, margin=50, w=1280, h=720)
#image = mpimg.imread("corrected_images/adj_test5.jpg")
#image = mpimg.imread("corrected_images/adj_straight_lines1.jpg")
image = mpimg.imread("debug_images/1260.jpg")

img_thres = road.image_threshold(image)
warped = road.perspective_transform(img_thres)
warped_orig = road.perspective_transform(image)
road.parse_image(image)
road.sobel_kernel=15
overlay, mapped_lanes = road.draw_overlay(image, warped)
show_grid((image, img_thres, overlay), ("Original", "Thresholded", "overlay"))
show_grid((warped_orig, warped, mapped_lanes), ("Original", "Thresholded", "Mapped Lanes"))
print(road.viewport)
print("Left",road.left_lane)
print("Right", road.right_lane)


In [ ]:
show_grid((warped_orig, warped), ("Original", "Thresholded"), ticks=(40,40))
show_grid((image, overlay), ("Original", "Overlay"))

In [2]:
import pickle
import imageio
imageio.plugins.ffmpeg.download()
from moviepy.editor import VideoFileClip
from IPython.display import HTML
from CameraOperations import correct_distortion
from LaneLines import Road
import matplotlib.image as mpimg

with open("./camera_parameters.p", mode='rb') as f:
    camera_params = pickle.load(f)
global_dist = camera_params['dist']
global_mtx = camera_params['mtx']


idx=0
cache = None
road = Road(window_height=80, window_width=100, margin=100, w=1280, h=720)
def make_frame_lane_markings(image):
    global idx
    global cache
    DEBUG = True
    un_dst = correct_distortion(image, global_mtx, global_dist)
    warped = road.parse_image(un_dst)
    lane_markings, mapped_lanes = road.draw_overlay(un_dst, warped)
    if DEBUG:
        mpimg.imsave("debug_images/challenge/{0}_overlay.jpg".format(idx), lane_markings)
        mpimg.imsave("debug_images/challenge/{0}_mapped_lanes.jpg".format(idx), mapped_lanes)
        mpimg.imsave("debug_images/challenge/{0}.jpg".format(idx), un_dst)
    idx = idx + 1
    return lane_markings

clip_output = "./marked_v9_challenge_video.mp4" #"marked_v6_challenge_video.mp4" # 
clip_handle = VideoFileClip("challenge_video.mp4", audio=False)
img_clip = clip_handle.fl_image(make_frame_lane_markings) 
%time img_clip.write_videofile(clip_output, audio=False)

TypeError: 'NoneType' object is not subscriptable